In [1]:
from sklearn.decomposition import PCA
from pennylane import numpy as np
import pennylane as qml
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from itertools import combinations
from scipy import stats as st
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

# Function Definitions and Quantum Device Setup

In [2]:
def layer(x, params, wires, i0=0, inc=1):
    """Building block of the embedding ansatz"""
    i = i0
    for j, wire in enumerate(wires):
        qml.Hadamard(wires=[wire])
        qml.RZ(x[i % len(x)], wires=[wire])
        i += inc
        qml.RY(params[0, j], wires=[wire])

    qml.broadcast(unitary=qml.CRZ, pattern="ring", wires=wires, parameters=params[1])

In [3]:
def ansatz(x, params, wires):
    """The embedding ansatz"""
    for j, layer_params in enumerate(params):
        layer(x, layer_params, wires, i0=j * len(wires))


adjoint_ansatz = qml.adjoint(ansatz)

In [4]:
def random_params(num_wires, num_layers):
    """Generate random variational parameters in the shape for the ansatz."""
    np.random.seed(5687)
    return np.random.uniform(0, 2 * np.pi, (num_layers, 2, num_wires), requires_grad=True)

In [5]:
dev = qml.device("default.qubit", wires=3, shots=None)
wires = dev.wires.tolist()

In [6]:
@qml.qnode(dev)
def kernel_circuit(x1, x2, params):
    ansatz(x1, params, wires=wires)
    adjoint_ansatz(x2, params, wires=wires)
    return qml.probs(wires=wires)

In [7]:
def kernel(x1, x2, params):
    return kernel_circuit(x1, x2, params)[0]

In [8]:
def accuracy(classifier, X, Y_target):
    return 1 - np.count_nonzero(classifier.predict(X) - Y_target) / len(Y_target)

In [9]:
def target_alignment(
    X,
    Y,
    kernel,
    assume_normalized_kernel=False,
    rescale_class_labels=True,
):
    """Kernel-target alignment between kernel and labels."""

    K = qml.kernels.square_kernel_matrix(
        X,
        kernel,
        assume_normalized_kernel=assume_normalized_kernel,
    )

    if rescale_class_labels:
        nplus = np.count_nonzero(np.array(Y) == 1)
        nminus = len(Y) - nplus
        _Y = np.array([y / nplus if y == 1 else y / nminus for y in Y])
    else:
        _Y = np.array(Y)

    T = np.outer(_Y, _Y)
    inner_product = np.sum(K * T)
    norm = np.sqrt(np.sum(K * K) * np.sum(T * T))
    inner_product = inner_product / norm

    return inner_product

In [10]:
def train(params, X_train, y_train, steps, subset_size, lr,alignment_target=None, idx=None):
    opt = qml.GradientDescentOptimizer(lr)

    for i in range(steps):
        # Choose subset of datapoints to compute the KTA on.
        if idx:
            subset = idx
        else:
            subset = np.random.choice(list(range(len(X_train))), subset_size)
        # Define the cost function for optimization
        cost = lambda _params: -target_alignment(
          X_train[subset],
          y_train[subset],
          lambda x1, x2: kernel(x1, x2, _params),
          assume_normalized_kernel=True,
        )
        # Optimization step
        params = opt.step(cost, params)

        # Report the alignment on the full dataset every 50 steps.
        if (i + 1) % 50 == 0:
            current_alignment = target_alignment(
              X_train,
              y_train,
              lambda x1, x2: kernel(x1, x2, params),
              assume_normalized_kernel=True,
            )
            print(f"Step {i+1} - Alignment = {current_alignment:.3f}")
            if alignment_target and (current_alignment >= alignment_target):
                break
    return params

In [25]:
from sklearn.datasets import load_digits
digits = load_digits()

In [12]:
X = digits['data']
Y = digits['target']

idx = np.hstack([np.where(Y==i)[0][:20] for i in np.unique(Y)])
X = X[idx]
Y = Y[idx]

pca = PCA(n_components=2)
X = pca.fit_transform(X)
scaler = MinMaxScaler()
X_minmax = scaler.fit_transform(X)*np.pi

In [15]:
quantum_accuracies_trained = []
quantum_accuracies_untrained = []
for rdm in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X_minmax,Y, test_size=0.3, stratify=Y, random_state=rdm)
    pred_all_trained = []
    pred_all_untrained = []
    all_params = {}
    init_params = random_params(num_wires=3, num_layers=4)
    init_kernel = lambda x1, x2: kernel(x1, x2, init_params)
    for y1, y2 in combinations(np.unique(Y).numpy(), r=2):
        mask = (y_train==y1) | (y_train==y2)
        y_train_pair = y_train[mask]
        X_train_pair = X_train[mask]
        y_train_pair [y_train_pair == y1] = -1
        y_train_pair [y_train_pair == y2] = 1
        params = all_params.get((y1, y2), init_params)
        new_kernel = lambda x1, x2: kernel(x1, x2, params)
        svm_untrained = SVC(kernel=lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, new_kernel)).fit(X_train_pair, y_train_pair)
        test_mask = (y_test==y1) | (y_test==y2)
        y_test_pair = y_test[test_mask]
        X_test_pair = X_test[test_mask]
        y_test_pair [y_test_pair == y1] = -1
        y_test_pair [y_test_pair == y2] = 1
        y_pred_pair = svm_untrained.predict(X_test_pair)
        test_accuracy = 1 - np.count_nonzero(y_pred_pair - y_test_pair) / len(y_test_pair)
        print(f"Test Accuracy:{test_accuracy*100:.2f}")
        lr = 2
        new_test_accuracy = test_accuracy
        while (new_test_accuracy != 1) and (new_test_accuracy <= test_accuracy) and (lr >= 0.5):
            if lr >=1 :
                iterations= 500
            else:
                iterations= 1000
            params = all_params.get((y1, y2), init_params)
            params = train(params, X_train_pair, y_train_pair, iterations, 4, lr)
            all_params[(y1, y2)] = params
            trained_kernel = lambda x1, x2: kernel(x1, x2, all_params[(y1, y2)])
            svm_trained = SVC(kernel=lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)).fit(X_train_pair, y_train_pair)
            y_pred_pair_new = svm_trained.predict(X_test_pair)
            new_test_accuracy = 1 - np.count_nonzero(y_pred_pair_new - y_test_pair) / len(y_test_pair)
            print(f"Test Accuracy:{new_test_accuracy*100:.2f}")
            lr /= 2
        untrained_predictions = svm_untrained.predict(X_test)
        pred_all_untrained.append((untrained_predictions, (y1,y2)))
        if new_test_accuracy <= test_accuracy:
            pred_all_trained.append((untrained_predictions, (y1,y2)))
        else:
            pred_all_trained.append((svm_trained.predict(X_test), (y1,y2)))
    y_pred_all = []
    for y_pred_pair, labels in pred_all_trained:
        y_pred_pair[y_pred_pair==-1]=0
        labels = np.array(labels).numpy()
        y_pred_all.append(labels[y_pred_pair])
    y_pred = st.mode(np.vstack(y_pred_all).numpy(),axis=0)[0]
    final_accuracy_trained = 1 - np.count_nonzero(y_pred - y_test) / len(y_test)
    quantum_accuracies_trained.append(final_accuracy_trained*100)
    print(f"Final Accuracy Trained {final_accuracy_trained*100:.2f}")
    y_pred_all = []
    for y_pred_pair, labels in pred_all_untrained:
        y_pred_pair[y_pred_pair==-1]=0
        labels = np.array(labels).numpy()
        y_pred_all.append(labels[y_pred_pair])
    y_pred = st.mode(np.vstack(y_pred_all).numpy(),axis=0)[0]
    final_accuracy_untrained = 1 - np.count_nonzero(y_pred - y_test) / len(y_test)
    quantum_accuracies_untrained.append(final_accuracy_untrained*100)
    print(f"Final Accuracy Untrained {final_accuracy_untrained*100:.2f}")

Test Accuracy:100.00
Test Accuracy:66.67
Step 50 - Alignment = 0.231
Step 100 - Alignment = 0.229
Step 150 - Alignment = 0.230
Step 200 - Alignment = 0.227
Step 250 - Alignment = 0.231
Step 300 - Alignment = 0.234
Step 350 - Alignment = 0.228
Step 400 - Alignment = 0.226
Step 450 - Alignment = 0.233
Step 500 - Alignment = 0.249
Test Accuracy:75.00
Test Accuracy:100.00
Test Accuracy:100.00
Test Accuracy:100.00
Test Accuracy:100.00
Test Accuracy:100.00
Test Accuracy:100.00
Test Accuracy:91.67
Step 50 - Alignment = 0.616
Step 100 - Alignment = 0.631
Step 150 - Alignment = 0.634
Step 200 - Alignment = 0.646
Step 250 - Alignment = 0.650
Step 300 - Alignment = 0.649
Step 350 - Alignment = 0.651
Step 400 - Alignment = 0.649
Step 450 - Alignment = 0.643
Step 500 - Alignment = 0.655
Test Accuracy:91.67
Step 50 - Alignment = 0.655
Step 100 - Alignment = 0.655
Step 150 - Alignment = 0.653
Step 200 - Alignment = 0.655
Step 250 - Alignment = 0.656
Step 300 - Alignment = 0.657
Step 350 - Alignment =

Step 200 - Alignment = 0.191
Step 250 - Alignment = 0.190
Step 300 - Alignment = 0.192
Step 350 - Alignment = 0.191
Step 400 - Alignment = 0.190
Step 450 - Alignment = 0.192
Step 500 - Alignment = 0.188
Step 550 - Alignment = 0.189
Step 600 - Alignment = 0.189
Step 650 - Alignment = 0.188
Step 700 - Alignment = 0.189
Step 750 - Alignment = 0.189
Step 800 - Alignment = 0.184
Step 850 - Alignment = 0.185
Step 900 - Alignment = 0.186
Step 950 - Alignment = 0.188
Step 1000 - Alignment = 0.188
Test Accuracy:83.33
Test Accuracy:100.00
Test Accuracy:100.00
Test Accuracy:100.00
Test Accuracy:100.00
Test Accuracy:100.00
Test Accuracy:100.00
Test Accuracy:100.00
Final Accuracy Trained 83.33
Final Accuracy Untrained 81.67
Test Accuracy:100.00
Test Accuracy:66.67
Step 50 - Alignment = 0.261
Step 100 - Alignment = 0.280
Step 150 - Alignment = 0.297
Step 200 - Alignment = 0.305
Step 250 - Alignment = 0.280
Step 300 - Alignment = 0.290
Step 350 - Alignment = 0.290
Step 400 - Alignment = 0.260
Step 45

Step 200 - Alignment = 0.626
Step 250 - Alignment = 0.625
Step 300 - Alignment = 0.628
Step 350 - Alignment = 0.629
Step 400 - Alignment = 0.629
Step 450 - Alignment = 0.627
Step 500 - Alignment = 0.625
Step 550 - Alignment = 0.629
Step 600 - Alignment = 0.630
Step 650 - Alignment = 0.629
Step 700 - Alignment = 0.631
Step 750 - Alignment = 0.629
Step 800 - Alignment = 0.630
Step 850 - Alignment = 0.629
Step 900 - Alignment = 0.630
Step 950 - Alignment = 0.632
Step 1000 - Alignment = 0.629
Test Accuracy:91.67
Test Accuracy:100.00
Test Accuracy:75.00
Step 50 - Alignment = 0.184
Step 100 - Alignment = 0.188
Step 150 - Alignment = 0.193
Step 200 - Alignment = 0.200
Step 250 - Alignment = 0.200
Step 300 - Alignment = 0.198
Step 350 - Alignment = 0.196
Step 400 - Alignment = 0.198
Step 450 - Alignment = 0.210
Step 500 - Alignment = 0.215
Test Accuracy:83.33
Test Accuracy:100.00
Test Accuracy:83.33
Step 50 - Alignment = 0.252
Step 100 - Alignment = 0.264
Step 150 - Alignment = 0.271
Step 200 

Step 250 - Alignment = 0.690
Step 300 - Alignment = 0.691
Step 350 - Alignment = 0.692
Step 400 - Alignment = 0.692
Step 450 - Alignment = 0.693
Step 500 - Alignment = 0.690
Test Accuracy:91.67
Step 50 - Alignment = 0.692
Step 100 - Alignment = 0.691
Step 150 - Alignment = 0.690
Step 200 - Alignment = 0.692
Step 250 - Alignment = 0.692
Step 300 - Alignment = 0.692
Step 350 - Alignment = 0.692
Step 400 - Alignment = 0.692
Step 450 - Alignment = 0.692
Step 500 - Alignment = 0.692
Step 550 - Alignment = 0.692
Step 600 - Alignment = 0.692
Step 650 - Alignment = 0.692
Step 700 - Alignment = 0.692
Step 750 - Alignment = 0.691
Step 800 - Alignment = 0.692
Step 850 - Alignment = 0.691
Step 900 - Alignment = 0.692
Step 950 - Alignment = 0.691
Step 1000 - Alignment = 0.691
Test Accuracy:91.67
Test Accuracy:91.67
Step 50 - Alignment = 0.576
Step 100 - Alignment = 0.602
Step 150 - Alignment = 0.611
Step 200 - Alignment = 0.618
Step 250 - Alignment = 0.617
Step 300 - Alignment = 0.621
Step 350 - Al

Step 500 - Alignment = 0.657
Test Accuracy:91.67
Step 50 - Alignment = 0.662
Step 100 - Alignment = 0.662
Step 150 - Alignment = 0.661
Step 200 - Alignment = 0.663
Step 250 - Alignment = 0.662
Step 300 - Alignment = 0.656
Step 350 - Alignment = 0.658
Step 400 - Alignment = 0.663
Step 450 - Alignment = 0.663
Step 500 - Alignment = 0.663
Test Accuracy:91.67
Step 50 - Alignment = 0.664
Step 100 - Alignment = 0.664
Step 150 - Alignment = 0.663
Step 200 - Alignment = 0.662
Step 250 - Alignment = 0.662
Step 300 - Alignment = 0.662
Step 350 - Alignment = 0.664
Step 400 - Alignment = 0.663
Step 450 - Alignment = 0.663
Step 500 - Alignment = 0.663
Step 550 - Alignment = 0.663
Step 600 - Alignment = 0.662
Step 650 - Alignment = 0.663
Step 700 - Alignment = 0.662
Step 750 - Alignment = 0.661
Step 800 - Alignment = 0.659
Step 850 - Alignment = 0.659
Step 900 - Alignment = 0.661
Step 950 - Alignment = 0.661
Step 1000 - Alignment = 0.663
Test Accuracy:91.67
Test Accuracy:91.67
Step 50 - Alignment = 

...

Step 300 - Alignment = 0.196
Step 350 - Alignment = 0.194
Step 400 - Alignment = 0.194
Step 450 - Alignment = 0.196
Step 500 - Alignment = 0.193
Test Accuracy:75.00
Step 50 - Alignment = 0.194
Step 100 - Alignment = 0.196
Step 150 - Alignment = 0.193
Step 200 - Alignment = 0.192
Step 250 - Alignment = 0.194
Step 300 - Alignment = 0.196
Step 350 - Alignment = 0.197
Step 400 - Alignment = 0.195
Step 450 - Alignment = 0.196
Step 500 - Alignment = 0.196
Step 550 - Alignment = 0.194
Step 600 - Alignment = 0.197
Step 650 - Alignment = 0.197
Step 700 - Alignment = 0.196
Step 750 - Alignment = 0.196
Step 800 - Alignment = 0.199
Step 850 - Alignment = 0.197
Step 900 - Alignment = 0.197
Step 950 - Alignment = 0.198
Step 1000 - Alignment = 0.199
Test Accuracy:83.33
Test Accuracy:100.00
Test Accuracy:100.00
Test Accuracy:100.00
Test Accuracy:100.00
Test Accuracy:91.67
Step 50 - Alignment = 0.435
Step 100 - Alignment = 0.430
Step 150 - Alignment = 0.430
Step 200 - Alignment = 0.449
Step 250 - Align

Step 100 - Alignment = 0.331
Step 150 - Alignment = 0.330
Step 200 - Alignment = 0.326
Step 250 - Alignment = 0.325
Step 300 - Alignment = 0.345
Step 350 - Alignment = 0.334
Step 400 - Alignment = 0.340
Step 450 - Alignment = 0.361
Step 500 - Alignment = 0.357
Test Accuracy:58.33
Step 50 - Alignment = 0.363
Step 100 - Alignment = 0.369
Step 150 - Alignment = 0.365
Step 200 - Alignment = 0.364
Step 250 - Alignment = 0.366
Step 300 - Alignment = 0.370
Step 350 - Alignment = 0.374
Step 400 - Alignment = 0.371
Step 450 - Alignment = 0.374
Step 500 - Alignment = 0.378
Test Accuracy:66.67
Test Accuracy:91.67
Step 50 - Alignment = 0.448
Step 100 - Alignment = 0.463
Step 150 - Alignment = 0.474
Step 200 - Alignment = 0.489
Step 250 - Alignment = 0.516
Step 300 - Alignment = 0.516
Step 350 - Alignment = 0.506
Step 400 - Alignment = 0.516
Step 450 - Alignment = 0.523
Step 500 - Alignment = 0.526
Test Accuracy:91.67
Step 50 - Alignment = 0.530
Step 100 - Alignment = 0.532
Step 150 - Alignment = 0

Step 350 - Alignment = 0.502
Step 400 - Alignment = 0.504
Step 450 - Alignment = 0.505
Step 500 - Alignment = 0.505
Test Accuracy:83.33
Step 50 - Alignment = 0.498
Step 100 - Alignment = 0.506
Step 150 - Alignment = 0.503
Step 200 - Alignment = 0.506
Step 250 - Alignment = 0.506
Step 300 - Alignment = 0.504
Step 350 - Alignment = 0.488
Step 400 - Alignment = 0.503
Step 450 - Alignment = 0.508
Step 500 - Alignment = 0.507
Test Accuracy:83.33
Step 50 - Alignment = 0.506
Step 100 - Alignment = 0.508
Step 150 - Alignment = 0.506
Step 200 - Alignment = 0.507
Step 250 - Alignment = 0.506
Step 300 - Alignment = 0.504
Step 350 - Alignment = 0.508
Step 400 - Alignment = 0.508
Step 450 - Alignment = 0.507
Step 500 - Alignment = 0.508
Step 550 - Alignment = 0.508
Step 600 - Alignment = 0.507
Step 650 - Alignment = 0.507
Step 700 - Alignment = 0.507
Step 750 - Alignment = 0.507
Step 800 - Alignment = 0.507
Step 850 - Alignment = 0.507
Step 900 - Alignment = 0.507
Step 950 - Alignment = 0.507
Step 

Step 150 - Alignment = 0.677
Step 200 - Alignment = 0.684
Step 250 - Alignment = 0.685
Step 300 - Alignment = 0.687
Step 350 - Alignment = 0.687
Step 400 - Alignment = 0.690
Step 450 - Alignment = 0.690
Step 500 - Alignment = 0.691
Test Accuracy:91.67
Step 50 - Alignment = 0.690
Step 100 - Alignment = 0.688
Step 150 - Alignment = 0.691
Step 200 - Alignment = 0.690
Step 250 - Alignment = 0.689
Step 300 - Alignment = 0.691
Step 350 - Alignment = 0.690
Step 400 - Alignment = 0.690
Step 450 - Alignment = 0.690
Step 500 - Alignment = 0.691
Test Accuracy:91.67
Step 50 - Alignment = 0.691
Step 100 - Alignment = 0.691
Step 150 - Alignment = 0.690
Step 200 - Alignment = 0.691
Step 250 - Alignment = 0.691
Step 300 - Alignment = 0.690
Step 350 - Alignment = 0.690
Step 400 - Alignment = 0.690
Step 450 - Alignment = 0.690
Step 500 - Alignment = 0.690
Step 550 - Alignment = 0.689
Step 600 - Alignment = 0.691
Step 650 - Alignment = 0.691
Step 700 - Alignment = 0.690
Step 750 - Alignment = 0.691
Step 

Step 50 - Alignment = 0.397
Step 100 - Alignment = 0.417
Step 150 - Alignment = 0.418
Step 200 - Alignment = 0.422
Step 250 - Alignment = 0.430
Step 300 - Alignment = 0.436
Step 350 - Alignment = 0.438
Step 400 - Alignment = 0.443
Step 450 - Alignment = 0.449
Step 500 - Alignment = 0.451
Test Accuracy:91.67
Test Accuracy:100.00
Test Accuracy:91.67
Step 50 - Alignment = 0.609
Step 100 - Alignment = 0.619
Step 150 - Alignment = 0.622
Step 200 - Alignment = 0.633
Step 250 - Alignment = 0.626
Step 300 - Alignment = 0.621
Step 350 - Alignment = 0.636
Step 400 - Alignment = 0.633
Step 450 - Alignment = 0.631
Step 500 - Alignment = 0.628
Test Accuracy:91.67
Step 50 - Alignment = 0.633
Step 100 - Alignment = 0.633
Step 150 - Alignment = 0.633
Step 200 - Alignment = 0.626
Step 250 - Alignment = 0.634
Step 300 - Alignment = 0.634
Step 350 - Alignment = 0.635
Step 400 - Alignment = 0.634
Step 450 - Alignment = 0.634
Step 500 - Alignment = 0.629
Test Accuracy:91.67
Step 50 - Alignment = 0.633
Step

In [18]:
for ker in ['rbf', 'linear', 'poly']:
  acc = []
  for rdm in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X_minmax,Y, test_size=0.3, stratify=Y, random_state=rdm)
    clf = SVC(kernel=ker)
    clf.fit(X_train, y_train)
    acc.append(accuracy(clf, X_test, y_test)*100)
  print(f"Kernel: {ker}\t Mean: {np.mean(acc):.2f} \t Std: {np.std(acc):.2f}")

Kernel: rbf	 Mean: 75.33 	 Std: 4.93
Kernel: linear	 Mean: 72.17 	 Std: 4.72
Kernel: poly	 Mean: 72.17 	 Std: 6.01


In [23]:
round(np.mean(quantum_accuracies_trained),2)

76.33

In [24]:
round(np.std(quantum_accuracies_trained),2)

5.57

In [21]:
round(np.mean(quantum_accuracies_untrained),2)

75.5

In [22]:
round(np.std(quantum_accuracies_untrained),2)

5.48